# Forward Propagation - Feed Forward Neural Network

### Import all library

In [1]:
import numpy as np
import json
import networkx as nx
import matplotlib.pyplot as plt

### Init activation function list

In [2]:
activation = [
    'relu',
    'sigmoid',
    'linear',
    'softmax',
]

### Init all functionality class to build Forward Propagation and FFNN

In [3]:
class ActivationFunction:
    def __init__(self, activation_function):
        if activation_function == 'sigmoid':
            self.name = 'sigmoid'
            self.function = lambda net: 1 / (1 + np.exp(-net))
            self.error_term_output = lambda output, target : output * (1 - output) * (target - output)
            self.error_term_hidden = lambda output, error_term_output, weights : output * (1 - output) * np.dot(error_term_output, weights.T)
            self.loss = lambda output, target : np.sum((target - output) ** 2) / 2
        elif activation_function == 'relu':
            self.name = 'relu'
            self.function = lambda net: np.maximum(0, net)
            self.error_term_output = lambda output, target : np.where(output > 0, 1, 0) * (target - output)
            self.error_term_hidden = lambda output, error_term_output, weights : np.where(output > 0, 1, 0) * np.dot(error_term_output, weights.T)
            self.loss = lambda output, target : np.sum((target - output) ** 2) / 2
        elif activation_function == 'linear':
            self.name = 'linear'
            self.function = lambda net: net
            self.error_term_output = lambda output, target : target - output
            self.error_term_hidden = lambda _, error_term_output, weights : np.dot(error_term_output, weights.T)
            self.loss = lambda output, target : np.sum((target - output) ** 2) / 2
        elif activation_function == 'softmax':
            self.name = 'softmax'
            self.function = lambda net: np.exp(net) / np.sum(np.exp(net))
            self.error_term_output = lambda output, target : (target - output)
            self.error_term_hidden = lambda _, error_term_output, weights : np.dot(error_term_output, weights.T)
            self.loss = lambda output, target : self.softmax_loss(output, target)

    def softmax_loss(self, output, target):
        target_label = np.argmax(target, axis=1)
        return -np.sum(np.log(output[np.arange(len(target_label)), target_label]))

    def get_name(self):
        return self.name
    
    def get_activation_function(self):
        return self.function
    
    def get_error_term_output(self):
        return self.error_term_output
    
    def get_error_term_hidden(self):
        return self.error_term_hidden
    
    def get_loss(self):
        return self.loss

In [4]:
class Layer:
    def __init__(self, neuron: int, activation_function: str, weights: np.array, bias: np.array, out: bool = False):
        self.neuron = neuron
        self.weights = weights
        self.bias = bias
        self.out = out
        if activation_function not in activation:
            raise Exception('Invalid activation function')
        else:
            self.activation_function = ActivationFunction(activation_function)
            self.function = self.activation_function.get_activation_function()
            if out:
                self.error_term = self.activation_function.get_error_term_output()
            else:
                self.error_term = self.activation_function.get_error_term_hidden()
            self.loss = self.activation_function.get_loss()

    def forward(self, input: np.array):
        self.last_activation = self.function(np.dot(input, self.weights) + self.bias)
        return self.last_activation

    def update_weight(self, input: np.array, error_term: np.array, learning_rate: float):
        self.weights += learning_rate * np.dot(input.T, error_term)
        self.bias += learning_rate * error_term.sum(axis=0)

In [5]:
class FFNN:
    def __init__(self):
        self.layers = []
        self.node_weights = []

    def add_layer(self, layer: Layer):
        self.layers.append(layer)

    def forward(self, input: np.array):
        output = input
        batch_count = len(output)
        for i in range(batch_count):
            self.node_weights.append([output[i]])
        for layer in self.layers:
            output = layer.forward(output)
            for i in range(batch_count):
                temp_nodes = output[i].tolist()
                self.node_weights[i].append(temp_nodes)
        return output

    def fit(self, input: np.array, target: np.array, learning_rate: float, batch_size: int, max_iteration: int, error_threshold: float):
        for iter in range(max_iteration):
            total_err = 0
            for i in range(0, len(input), batch_size):
                input_batch = input[i:i+batch_size]
                target_batch = target[i:i+batch_size]
                output_batch = self.forward(input_batch)
                error_batch = np.mean(self.layers[-1].loss(output_batch, target_batch))
                total_err += error_batch

                self.backward(input_batch, output_batch, target_batch, learning_rate)
            total_err /= len(input) / batch_size
            if np.abs(total_err) < error_threshold:
                return "error_threshold"
        return "max_iteration"

    def backward(self, input : np.array, output: np.array, target: np.array, learning_rate: float):
        first_input = input
        for i in range(len(self.layers)-1, -1, -1):
            layer = self.layers[i]
            input = first_input if i == 0 else self.layers[i-1].last_activation
            if i == len(self.layers)-1:
                error_term = layer.error_term(output, target)
                layer.update_weight(input, error_term, learning_rate)
            else:
                error_term_output = self.layers[i+1].error_term(self.layers[i+1].last_activation, target)
                error_term = layer.error_term(self.layers[i].last_activation, error_term_output, self.layers[i+1].weights)
                layer.update_weight(input, error_term, learning_rate)
        self.node_weights = []

In [6]:
class NeuralNetworkGraph:
    def __init__(self):
        self.graphs = nx.DiGraph()
        self.node_labels = {}

    def add_nodes_layer(self, nodes, layer):
        self.graphs.add_nodes_from(nodes, layer=layer)

    def add_edge(self, node_source, node_goal):
        self.graphs.add_edge(node_source, node_goal)

    def add_all_nodes(self, node_weights):

        # Add input bias label
        self.node_labels[0] = "Input"+str(0)+": "+str(1)

        node_number = 1

        self.add_nodes_layer([i for i in range(len(node_weights[0])+1)], 0)

        # Add input label
        for i, node_value in enumerate(node_weights[0]):
            self.node_labels[node_number] = "Input"+str(i+1)+": "+str(node_value)
            node_number += 1

        # Add nodes for each layer with the subset key
        for i in range(1, len(node_weights)-1):
            self.node_labels[node_number] = "H"+str(i)+str(0)+": "+str(1)
            self.add_nodes_layer([node_number+j for j in range(len(node_weights[i])+1)], i)
            node_number += 1
            for j in range(len(node_weights[i])):
                self.node_labels[node_number] = "H"+str(i)+str(j+1)+": "+str(node_weights[i][j])
                node_number += 1

        self.add_nodes_layer([node_number+j for j in range(len(node_weights[-1]))], len(node_weights))

        # Add output nodes
        for i, node_value in enumerate(node_weights[-1]):
            self.node_labels[node_number] = "Output"+str(i+1)+": "+str(node_value)
            node_number += 1

    def add_all_edges(self, node_weights):
        number_of_prev_neuron = 0
        curr_number_neuron = 0

        # Add edge for each node of input layer and hidden layer
        for layer_number in range(len(node_weights)-2):
            curr_number_neuron += len(node_weights[layer_number])+1
            for i in range(len(node_weights[layer_number])+1):
                for j in range(len(node_weights[layer_number+1])):
                    self.add_edge(number_of_prev_neuron, curr_number_neuron+j+1)
                number_of_prev_neuron += 1

        # Add edge for each node on output layer
        curr_number_neuron += len(node_weights[-2])+1
        for i in range(len(node_weights[-2])+1):
            for j in range(len(node_weights[-1])):
                self.add_edge(number_of_prev_neuron, curr_number_neuron+j)
            number_of_prev_neuron += 1

In [7]:
# for saving the model
def saveModel(weights, layers, fileName):
    case_dict = {'case': {'weights': weights, 'model': {'layers': layers}}}
    with open(f"../../Bagian-B/model/{fileName}_latest_weights_and_structures.json", 'w') as outfile:
        json.dump(case_dict, outfile, indent=4)

In [8]:
# for visualizing the model
def visualizeModel(ffnn, weights):
    # Initialize a directed graph for visualization
    node_weights = ffnn.node_weights
    batch_count = len(node_weights)
    for batch_num in range(batch_count):
        neural_network_graph = NeuralNetworkGraph()

        neural_network_graph.add_all_nodes(node_weights[batch_num])

        neural_network_graph.add_all_edges(node_weights[batch_num])

        # Assuming neural_network_graph.graphs is your graph object
        neural_network_graphs = neural_network_graph.graphs  # Assuming this is a correct reference

        # Add 'layer' attribute to nodes if it's missing
        for node in neural_network_graphs.nodes():
            if 'layer' not in neural_network_graphs.nodes[node]:
                neural_network_graphs.nodes[node]['layer'] = 0  # Set a default layer if needed

        # Plot the neural network structure
        pos = nx.multipartite_layout(neural_network_graphs, subset_key="layer", align='horizontal')
        nx.draw(neural_network_graphs, pos, with_labels=True, labels=neural_network_graph.node_labels, node_size=2000, node_color="lightblue", font_size=7, font_weight="bold")

        # Add edge labels for better understanding
        edge_labels = {}
        index = 0
        sub_index = 0
        sub_sub_index = 0
        for u, v in neural_network_graphs.edges():
            edge_labels[(u,v)] = "W: "+str(weights[index][sub_index][sub_sub_index])
            if(sub_sub_index+1<len(weights[index][sub_index])):
                sub_sub_index += 1
            else:
                sub_sub_index = 0
                if(sub_index+1<len(weights[index])):
                    sub_index += 1
                else:
                    sub_index = 0
                    index += 1
        print("==========================================================================================================================================================================================================================")
        print("Edge labels: ",edge_labels)
        print("==========================================================================================================================================================================================================================")
        nx.draw_networkx_edge_labels(neural_network_graphs, pos, edge_labels=edge_labels, font_size=7, font_color='red')

        plt.title("Neural Network Structure (Input to Output)")
        plt.axis('off')
        plt.show()

In [9]:
# without using saved model (contains weights and structure)
def calculateWithoutSavedModel(fileName):
    model = open(f'../../Bagian-A/test/{fileName}.json', 'r')
    model = json.load(model)

    layers = model['case']['model']['layers']
    weights = model['case']['weights']

    # for saving model
    layers_dict = []
    weights_dict = []

    ffnn = FFNN()
    for i in range (len(layers)):
        layer = layers[i]
        weight = weights[i]

        # add the layers and weights
        layers_dict.append(layer)
        weights_dict.append(weight)

        ffnn.add_layer(Layer(layer["number_of_neurons"], layer["activation_function"], np.array(weight[1:]), np.array(weight[0])))

    saveModel(weights_dict, layers_dict, fileName) # save the model

    input = model["case"]["input"]

    output = ffnn.forward(input).tolist()
    expected_output = model['expect']['output']
    print("==========================================================================================================================================================================================================================")
    print("Node values: ", ffnn.node_bobots)

    print(f'output: {output}')
    print(f'expected output: {expected_output}')
    print("==========================================================================================================================================================================================================================")

    visualizeModel(ffnn, weights)

In [10]:
# with saved model
def calculateWithSavedModel(fileName):
    model = open(f'../../Bagian-A/test/{fileName}.json', 'r')
    model = json.load(model)

    # loading the saved models (it contains the layers and weights)
    savedModel = open(f'../../Bagian-A/model/{fileName}_latest_weights_and_structures.json', 'r')
    savedModel = json.load(savedModel)

    layers = savedModel['case']['model']['layers']
    weights = savedModel['case']['weights']

    ffnn = FFNN()
    for i in range (len(layers)):
        layer = layers[i]
        weight = weights[i]
        ffnn.add_layer(Layer(layer["number_of_neurons"], layer["activation_function"], np.array(weight[1:]), np.array(weight[0])))

    input = model["case"]["input"]

    output = ffnn.forward(input).tolist()
    expected_output = model['expect']['output']
    print("==========================================================================================================================================================================================================================")
    print("Node values: ", ffnn.node_weights)
    print(f'output: {output}')
    print(f'expected output: {expected_output}')
    print("==========================================================================================================================================================================================================================")

    visualizeModel(ffnn, weights)

In [11]:
# region BAGIAN B
BAGIAN_B_TEST_CASES = {
    '1': '../../Bagian-B/test/linear_small_lr.json',
    '2': '../../Bagian-B/test/linear_two_iteration.json',
    '3': '../../Bagian-B/test/linear.json',
    '4': '../../Bagian-B/test/mlp.json',
    '5': '../../Bagian-B/test/relu_b.json',
    '6': '../../Bagian-B/test/softmax.json',
    '7': '../../Bagian-B/test/softmax_two_layer.json',
}

In [12]:
def start_test_case_b(test_case):
    model = open(test_case, 'r')
    model = json.load(model)

    layers = model['case']['model']['layers']
    init_weights = model['case']['initial_weights']

    ffnn = FFNN()
    for i in range (len(layers)):
        layer = layers[i]
        weight = init_weights[i]
        if (i == len(layers) - 1):
            ffnn.add_layer(Layer(layer["number_of_neurons"], layer["activation_function"], np.array(weight[1:]), np.array(weight[0]), out=True))
        else:
            ffnn.add_layer(Layer(layer["number_of_neurons"], layer["activation_function"], np.array(weight[1:]), np.array(weight[0])))


    input = np.array(model["case"]["input"])
    target = np.array(model["case"]["target"])

    params = model["case"]["learning_parameters"]

    stop_cond = ffnn.fit(input, target, params["learning_rate"], params["batch_size"], params["max_iteration"], params["error_threshold"])

    excepted_weights = model["expect"]["final_weights"]
    excepted_stop_cond = model["expect"]["stopped_by"]

    print(f'\nResults ------------------')
    print(f'Stop condition: {stop_cond}')
    print(f'Expected stop condition: {excepted_stop_cond}')

    for i in range(len(ffnn.layers)):
        print(f'Layer {i+1} weights:')
        bias_weights = np.array([ffnn.layers[i].bias] + ffnn.layers[i].weights.tolist())
        print(bias_weights)
        print(f'Expected Layer {i+1} weights:')
        print(np.array(excepted_weights[i]))
    print()
    visualizeModel(ffnn, init_weights)

In [13]:
def test_backward(test_case):

    l = len(BAGIAN_B_TEST_CASES) + 1
    print(f"Test Case: ", BAGIAN_B_TEST_CASES[test_case])

    if test_case == str(l):
        test_case = input("Input test file path: ")
        start_test_case_b(test_case)
        return
    
    start_test_case_b(BAGIAN_B_TEST_CASES[test_case])

# Kasus Uji

## Linear_small_lr

In [14]:
test_backward("1")

Test Case:  ../../Bagian-B/test/linear_small_lr.json

Results ------------------
Stop condition: max_iteration
Expected stop condition: max_iteration
Layer 1 weights:
[[ 0.1012  0.3006  0.1991]
 [ 0.4024  0.201  -0.7019]
 [ 0.1018 -0.799   0.4987]]
Expected Layer 1 weights:
[[ 0.1008  0.3006  0.1991]
 [ 0.402   0.201  -0.7019]
 [ 0.101  -0.799   0.4987]]



## linear_two_iteration

In [15]:
test_backward("2")

Test Case:  ../../Bagian-B/test/linear_two_iteration.json

Results ------------------
Stop condition: max_iteration
Expected stop condition: max_iteration
Layer 1 weights:
[[ 0.166  0.338  0.153]
 [ 0.502  0.226 -0.789]
 [ 0.214 -0.718  0.427]]
Expected Layer 1 weights:
[[ 0.166  0.338  0.153]
 [ 0.502  0.226 -0.789]
 [ 0.214 -0.718  0.427]]



## linear

In [16]:
test_backward("3")

Test Case:  ../../Bagian-B/test/linear.json

Results ------------------
Stop condition: max_iteration
Expected stop condition: max_iteration
Layer 1 weights:
[[ 0.22  0.36  0.11]
 [ 0.64  0.3  -0.89]
 [ 0.28 -0.7   0.37]]
Expected Layer 1 weights:
[[ 0.22  0.36  0.11]
 [ 0.64  0.3  -0.89]
 [ 0.28 -0.7   0.37]]



## mlp

In [17]:
test_backward("4")

Test Case:  ../../Bagian-B/test/mlp.json

Results ------------------
Stop condition: max_iteration
Expected stop condition: max_iteration
Layer 1 weights:
[[ 0.08581778  0.32009219]
 [-0.34196319  0.46252925]
 [ 0.45330896  0.441397  ]]
Expected Layer 1 weights:
[[ 0.08592   0.32276 ]
 [-0.33872   0.46172 ]
 [ 0.449984  0.440072]]
Layer 2 weights:
[[ 0.2748    0.188   ]
 [ 0.435904 -0.53168 ]
 [ 0.68504   0.7824  ]]
Expected Layer 2 weights:
[[ 0.2748    0.188   ]
 [ 0.435904 -0.53168 ]
 [ 0.68504   0.7824  ]]



## relu_b

In [18]:
test_backward("5")

Test Case:  ../../Bagian-B/test/relu_b.json

Results ------------------
Stop condition: max_iteration
Expected stop condition: max_iteration
Layer 1 weights:
[[-0.211   0.105   0.885 ]
 [ 0.3033  0.5285  0.3005]
 [-0.489  -0.905   0.291 ]]
Expected Layer 1 weights:
[[-0.211   0.105   0.885 ]
 [ 0.3033  0.5285  0.3005]
 [-0.489  -0.905   0.291 ]]



## softmax

In [19]:
test_backward("6")

Test Case:  ../../Bagian-B/test/softmax.json

Results ------------------
Stop condition: max_iteration
Expected stop condition: max_iteration
Layer 1 weights:
[[ 0.12674605  0.9149538  -0.14169985]
 [-0.33551647  0.67700488  0.45851159]
 [ 0.48314436 -0.85241216  0.2692678 ]
 [ 0.3400255   0.57237542 -0.31240092]
 [ 0.31397716  0.46349737  0.72252547]
 [-0.69652442  0.4789189   0.61760552]
 [-0.50884515 -0.36354141  0.57238656]
 [ 0.41891295  0.26354517 -0.48245812]
 [ 0.90374164 -0.01759501 -0.08614663]]
Expected Layer 1 weights:
[[ 0.12674605  0.9149538  -0.14169985]
 [-0.33551647  0.67700488  0.45851159]
 [ 0.48314436 -0.85241216  0.2692678 ]
 [ 0.3400255   0.57237542 -0.31240092]
 [ 0.31397716  0.46349737  0.72252547]
 [-0.69652442  0.4789189   0.61760552]
 [-0.50884515 -0.36354141  0.57238656]
 [ 0.41891295  0.26354517 -0.48245812]
 [ 0.90374164 -0.01759501 -0.08614663]]



## softmax_two_layer

In [20]:
test_backward("7")

Test Case:  ../../Bagian-B/test/softmax_two_layer.json

Results ------------------
Stop condition: error_threshold
Expected stop condition: error_threshold
Layer 1 weights:
[[-0.11927308  0.01473058 -0.47644775  0.42651942]
 [-0.70465145 -1.3477555  -1.56195381  0.7022227 ]
 [-0.4034573   1.65264803 -0.98739569 -1.31705963]]
Expected Layer 1 weights:
[[-0.28730211 -0.28822282 -0.70597451  0.42094471]
 [-0.5790794  -1.1836444  -1.34287961  0.69575311]
 [-0.41434377  1.51314676 -0.97649086 -1.3043465 ]]
Layer 2 weights:
[[-1.71196389  1.73196389]
 [-0.46000472  0.44000472]
 [ 1.22762769 -1.20762769]
 [-1.1009413   1.0809413 ]
 [ 1.07528479 -1.05528479]]
Expected Layer 2 weights:
[[-1.72078607  1.74078607]
 [-0.50352956  0.48352956]
 [ 1.25764816 -1.23764816]
 [-1.16998784  1.14998784]
 [ 1.0907634  -1.0707634 ]]



## MLP sklearn pada iris.csv

In [21]:
import pandas as pd
from sklearn.neural_network import MLPClassifier

data = pd.read_csv('../test/iris.csv')
training = data.drop(columns=['Species', 'Id'])
target = data['Species']

mlp = MLPClassifier(hidden_layer_sizes=(4, 9, 3), max_iter=1000, activation='logistic', learning_rate='constant', learning_rate_init=0.01)
mlp.fit(training, target)

target_predicted = mlp.predict(training)

print("Value comparison:")
print("Predicted values, Real values")
for i in range(len(target_predicted)):
    print(target_predicted[i], ",", target[i])
print("Accuracy:", mlp.score(training, target))

Value comparison:
Predicted values, Real values
Iris-versicolor , Iris-setosa
Iris-versicolor , Iris-setosa
Iris-versicolor , Iris-setosa
Iris-versicolor , Iris-setosa
Iris-versicolor , Iris-setosa
Iris-versicolor , Iris-setosa
Iris-versicolor , Iris-setosa
Iris-versicolor , Iris-setosa
Iris-versicolor , Iris-setosa
Iris-versicolor , Iris-setosa
Iris-versicolor , Iris-setosa
Iris-versicolor , Iris-setosa
Iris-versicolor , Iris-setosa
Iris-versicolor , Iris-setosa
Iris-versicolor , Iris-setosa
Iris-versicolor , Iris-setosa
Iris-versicolor , Iris-setosa
Iris-versicolor , Iris-setosa
Iris-versicolor , Iris-setosa
Iris-versicolor , Iris-setosa
Iris-versicolor , Iris-setosa
Iris-versicolor , Iris-setosa
Iris-versicolor , Iris-setosa
Iris-versicolor , Iris-setosa
Iris-versicolor , Iris-setosa
Iris-versicolor , Iris-setosa
Iris-versicolor , Iris-setosa
Iris-versicolor , Iris-setosa
Iris-versicolor , Iris-setosa
Iris-versicolor , Iris-setosa
Iris-versicolor , Iris-setosa
Iris-versicolor , Iris